In [ ]:
# 安装必要的Python库
# unsloth: 一个能将模型微调速度提升2倍并减少显存占用的库
# vllm: 一个用于快速高效LLM推理和服务的库，Unsloth在GRPO中会使用它来加速生成过程
pip install unsloth==2025.3.19 vllm==0.8.2

In [ ]:
from unsloth import FastLanguageModel
import torch

# 设置模型的最大序列长度。可以根据需要增加此值以支持更长的推理和上下文。
max_seq_length = 2048

# 设置LoRA的秩（rank）。秩越高，模型可能变得更“智能”，但训练和推理的速度会变慢，显存占用也会增加。
# LoRA是一种参数高效微调技术，通过训练小型的“适配器”矩阵来调整模型，而不是训练全部参数。
lora_rank = 32

# 从HuggingFace Hub加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    # 指定要加载的预训练模型名称。可以是HuggingFace官方模型、本地模型或Unsloth优化后的模型。
    model_name="Qwen/Qwen3-8B",
    # 设置模型的最大序列长度，与上面定义的变量一致。
    max_seq_length=max_seq_length,
    # 是否以4位精度加载模型。对于使用LoRA进行16位浮点数训练，此项应设置为False。
    load_in_4bit=False,
    # 是否启用vLLM进行快速推理。GRPO训练中生成多个响应时，此选项能显著提速。
    fast_inference=True,
    # 设置LoRA的最大秩，与上面定义的变量一致。
    max_lora_rank=lora_rank,
    # 设置GPU显存的使用率。如果遇到显存不足（OOM）的错误，可以适当降低此值。
    gpu_memory_utilization=0.7,
)

# 为模型添加PEFT（Parameter-Efficient Fine-Tuning，参数高效微调）配置，这里使用LoRA。
model = FastLanguageModel.get_peft_model(
    model,
    # LoRA的秩(r)，选择任何大于0的数字。建议值为 8, 16, 32, 64, 128。
    r=lora_rank,
    # target_modules 是一个列表，包含要应用LoRA技术的目标模块名称。
    # 通常我们会选择注意力机制中的投影层和前馈网络中的层。
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # 注意力机制中的查询、键、值、输出投影
        "gate_proj", "up_proj", "down_proj",     # 前馈网络中的门控、上行和下行投影
    ],
    # LoRA的alpha参数，通常设置为秩(r)的2倍，这是一种常见的做法，有助于稳定训练。
    lora_alpha=lora_rank * 2,
    # 是否使用梯度检查点技术。'unsloth'表示使用Unsloth的优化版本，可以显著减少训练时的显存占用。
    use_gradient_checkpointing="unsloth",
    # 设置随机种子，以确保实验结果的可复现性。
    random_state=3407,
)

In [ ]:
# 定义一些特殊的字符串标记，用于指导模型生成我们想要的格式。
# 这是一种“格式提示”或“模板化”的方法，让模型学会生成带有思考过程和最终答案的结构化输出。

# <start_working_out> 和 <end_working_out> 用于包裹模型的“思考过程”或“解题步骤”。
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"

# <SOLUTION> 和 </SOLUTION> 用于包裹模型给出的最终、简洁的答案。
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

# 定义系统提示（System Prompt）。这个提示会作为对话的初始指令，告诉模型它的角色和任务。
# 在这里，我们要求模型先进行思考，将过程放在<start_working_out>和<end_working_out>之间，
# 然后再将最终答案放在<SOLUTION>和</SOLUTION>之间。
system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""

# 打印系统提示，查看其内容。
system_prompt

In [ ]:
# 创建一个自定义的聊天模板（Chat Template）。
# 聊天模板使用Jinja2语法，定义了如何将多轮对话（包含system, user, assistant等角色）格式化为单个字符串，
# 以便输入给模型进行训练或推理。
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# 将模板中的占位符替换为我们之前定义的特定字符串。
# 这样做可以使模板适应我们自定义的格式要求。
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")

# 将我们创建的自定义聊天模板赋值给分词器（tokenizer）的chat_template属性。
# 这样，之后调用tokenizer.apply_chat_template时，就会使用这个新模板。
tokenizer.chat_template = chat_template

In [ ]:
# 使用分词器的apply_chat_template方法来测试我们自定义的聊天模板。
# 输入是一个包含多轮对话的列表，每轮对话是一个字典，包含'role'和'content'。
# tokenize = False 表示我们只想看到格式化后的字符串，而不是token ID。
# add_generation_prompt = True 会在末尾添加生成提示，这里是我们定义的'{reasoning_start}'，
# 引导模型从“思考过程”开始生成回答。
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

# 从HuggingFace Hub加载'unsloth/OpenMathReasoning-mini'数据集，并选择'cot' (Chain of Thought)子集。
dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")

# 将数据集转换为pandas DataFrame，并只保留我们需要的列。
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# 数据清洗：过滤数据集，只保留'expected_answer'列是数值的行。
# 首先，尝试将'expected_answer'列转换为数值类型，无法转换的将变为NaN (Not a Number)。
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# 然后，根据is_number布尔序列，筛选出值为数值的行。
dataset = dataset.iloc[np.where(is_number)[0]]

# 显示处理后的数据集，查看其结构和内容。
dataset

In [ ]:
# 定义一个函数，用于将数据集中的每一行格式化为我们需要的对话格式。
def format_dataset(x):
    # 提取原始数据列
    expected_answer = x["expected_answer"] # 期望的答案
    problem = x["problem"] # 问题本身

    # 清理模型生成的解决方案，移除其中已有的<think>和</think>标签。
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # 移除思考过程文本两端的空白字符（如换行符和空格）。
    thoughts = thoughts.strip()

    # 使用我们自定义的格式，将思考过程和最终答案组合成一个字符串。
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    
    # 返回一个列表，其中包含符合聊天模板输入格式的字典。
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

# 使用.apply方法将format_dataset函数应用到DataFrame的每一行，并将结果存储在新列'Messages'中。
dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

In [ ]:
# 再次调用apply_chat_template，这次应用于处理后的数据集的第一行，
# 以验证格式化是否正确。
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

In [ ]:
# 计算每条格式化后的消息被分词器处理后得到的token数量，并将其存储在新列'N'中。
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

# 过滤数据集，只保留token数量小于等于max_seq_length/2的样本。
# 这样做是为了确保在训练时，输入（prompt）不会太长，能给模型的生成（completion）留下足够的空间，
# 避免因超出最大序列长度而被截断。
dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()

# 显示过滤后数据集的形状（行数，列数）。
dataset.shape

In [ ]:
from datasets import Dataset

# 将所有格式化好的消息（Messages列）应用聊天模板，生成最终的训练文本，并存储在'text'列中。
dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)

# 将pandas DataFrame转换回HuggingFace的Dataset对象，这是SFTTrainer所要求的格式。
dataset = Dataset.from_pandas(dataset)

# 显示最终准备好的Dataset对象信息。
dataset

In [ ]:
from trl import SFTTrainer, SFTConfig

# 实例化SFTTrainer，用于进行有监督微调。
trainer = SFTTrainer(
    model = model, # 已经加载并添加了LoRA适配器的模型
    tokenizer = tokenizer, # 配套的分词器
    train_dataset = dataset, # 训练数据集
    args = SFTConfig( # SFT训练的配置参数
        # 指定数据集中包含完整训练文本的列名。
        dataset_text_field = "text",
        # 每个设备上的训练批次大小。
        per_device_train_batch_size = 1,
        # 梯度累积步数。实际的批次大小 = per_device_train_batch_size * gradient_accumulation_steps。
        # 用于在显存有限的情况下模拟更大的批次大小。
        gradient_accumulation_steps = 1,
        # 预热步数，在训练初期使用较小的学习率，有助于稳定训练。
        warmup_steps = 5,
        # 训练的总轮数（epochs）。
        num_train_epochs = 2,
        # 学习率。对于较长时间的训练，可以适当减小此值，例如2e-5。
        learning_rate = 2e-4,
        # 每隔多少步记录一次日志。
        logging_steps = 5,
        # 优化器类型。'adamw_8bit'是一种内存效率更高的AdamW优化器。
        optim = "adamw_8bit",
        # 权重衰减，一种正则化技术，防止过拟合。
        weight_decay = 0.01,
        # 学习率调度器类型，'linear'表示学习率会从初始值线性衰减到0。
        lr_scheduler_type = "linear",
        # 随机种子，保证训练过程的可复现性。
        seed = 3407,
        # 将训练日志报告到指定的平台，如'wandb'（Weights & Biases）或'tensorboard'。
        # 'swanlab'是一个类似的实验跟踪工具。
        report_to = "swanlab",
    ),
)

In [ ]:
# 开始SFT训练过程。
trainer.train()

In [ ]:
# SFT训练后进行推理测试
# 从数据集中取出一条样本的前两部分（system和user消息）作为推理的输入。
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # 必须添加此项以触发模型的生成
)

from transformers import TextStreamer

# 使用model.generate进行推理
_ = model.generate(
    # 将文本输入转换为PyTorch张量，并移动到CUDA设备上。
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    # temperature=0 表示使用贪心解码，选择概率最高的token，生成结果更具确定性。
    temperature = 0,
    # 设置生成的最大新token数量。
    max_new_tokens = 1024,
    # 使用TextStreamer可以流式输出结果，即逐个token地打印生成内容，而不是等全部生成完再输出。
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

In [ ]:
# --- GRPO 训练部分 --- #
# 为了进行GRPO训练，我们首先清理内存，为加载新数据集做准备。

# 删除之前的数据集对象
del dataset
# 清空PyTorch的CUDA缓存，释放未被引用的GPU内存
torch.cuda.empty_cache()
# 导入垃圾回收模块并执行垃圾回收
import gc
gc.collect()

In [ ]:
# 为GRPO训练加载一个新的、更大的数学推理数据集
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

In [ ]:
# 查看新数据集的一条样本的问题（prompt）
dataset[0]["prompt"]

In [ ]:
# 查看该样本对应的解决方案（solution）
dataset[0]["solution"]

In [ ]:
# 定义一个函数来提取答案。在这个数据集中，答案直接就是solution字段，
# 但在其他数据集中（如GSM8K），答案可能被####标记包围，这个函数是为此类情况准备的。
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text

extract_hash_answer(dataset[0]["solution"])

In [ ]:
# 再次对新数据集进行格式化，以符合我们的对话模板。
# 这次我们只准备system和user角色的消息，assistant部分将由模型在GRPO训练中生成。
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    # 同时提取出标准答案，用于后续的奖励函数评估。
    "answer": extract_hash_answer(x["solution"]),
})

# 查看格式化后的第一条数据。
dataset[0]

In [ ]:
# --- 定义GRPO的奖励函数 --- #
# 奖励函数是GRPO的核心，它评估模型生成的回答的质量，并返回一个分数。
# 分数越高，表示生成的回答越好。

import re

# 奖励函数1: 精确格式匹配
# 我们定义一个正则表达式来检查模型的输出是否严格遵循了 <end...><SOLUTION>...</SOLUTION> 的格式。

# 首先，创建一个正则表达式片段，用于匹配可能存在或不存在的EOS（end-of-sentence）令牌。
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

# 编译完整的正则表达式
match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

In [ ]:
# 测试正则表达式能否成功从一个符合格式的字符串中提取出答案 '2'。
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

In [ ]:
# 再次测试，即使答案前后有空格和换行符，也应该能正确提取。
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

In [ ]:
# 奖励函数2: 近似格式匹配
# 这个函数不检查答案是否正确，只检查格式。它计算输出中包含了多少个我们定义的特殊标签。
# 如果模型生成了所有必需的标签（每个一次），它会得到正分；否则会得到负分。
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        
        # 计算每个标签出现的次数，如果恰好是1次，则加分，否则扣分。
        # 无需奖励<start_working_out>，因为我们总是在提示中预先添加它。
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

In [ ]:
# 奖励函数3: 检查答案是否正确（字符串匹配）
# 这个函数使用我们之前定义的正则表达式提取模型给出的答案，并与标准答案进行比较。
def check_answer(prompts, completions, answer, **kwargs):
    # 获取用户的问题和模型生成的所有回答
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    # 使用正则表达式从每个回答中提取出<SOLUTION>标签内的内容
    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # 遍历每个提取出的答案和对应的标准答案
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        # 如果没有提取到答案（格式错误），给予重罚。
        if guess is None:
            scores.append(-2.0)
            continue
        # 如果答案完全正确，给予最高分！
        if guess == true_answer:
            score += 5.0
        # 如果去除空格后答案正确，也给予较高分数。
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # 对于数值答案，我们也可以奖励近似正确的答案。
            # 如果答案在真实答案的±10%范围内，给予奖励。
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # 错误答案给予惩罚
            except:
                # 如果无法转换为浮点数，给予重罚。
                score -= 4.5
        scores.append(score)
    return scores

In [ ]:
# 奖励函数4: 检查答案是否为数值（更宽松的数值匹配）
# 定义一个新的正则表达式，专门用于从<SOLUTION>标签中提取数值。
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
# 测试正则表达式
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

In [ ]:
# 定义全局变量，用于控制日志打印的频率。
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

# 奖励函数4的实现
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    # 使用新的正则表达式提取数值
    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # 为了便于调试，每隔几步打印一次问题、答案、模型响应和提取结果。
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        # 如果没有提取到数值，给予重罚。
        if guess is None:
            scores.append(-2.5)
            continue
        # 尝试将提取的字符串和标准答案都转换为浮点数进行比较。
        try:
            true_answer = float(true_answer.strip())
            # 移除逗号，如 '123,456' -> '123456'
            guess       = float(guess.strip().replace(",", ""))
            # 如果数值完全相等，给予高分，否则给予惩罚。
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            # 如果转换失败，不给分也不扣分。
            scores.append(0)
            continue
    return scores

In [ ]:
# 再次对数据集进行预处理，为GRPO训练做准备

# 将数据集的prompt部分（system和user消息）分词，并存储token
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
# 打印第一条数据分词后解码的结果，进行验证
print(tokenizer.decode(tokenized[0]["tokens"]))
# 计算每条prompt的token长度
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
# 计算所有prompt长度的90%分位数，以此作为最大prompt长度的参考。
# 这样可以过滤掉极少数过长的prompt，使训练更稳定高效。
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# 根据计算出的最大长度，过滤数据集。
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

In [ ]:
# 设置GRPO训练的参数

# prompt的最大长度，留一个token的余量
max_prompt_length = maximum_length + 1
# completion（模型生成部分）的最大长度
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
# 配置vLLM的采样参数，这些参数在GRPO训练中用于从模型生成多个不同的回答。
vllm_sampling_params = SamplingParams(
    min_p = 0.1, # 最小概率采样（Min-P），忽略概率低于此值的token
    top_p = 1.0, # Top-P（核）采样，从累积概率超过p的最小token集合中采样
    top_k = -1,  # Top-K采样，-1表示不启用
    seed = 3407, # 采样种子，保证可复现性
    stop = [tokenizer.eos_token], # 遇到EOS令牌时停止生成
    include_stop_str_in_output = True, # 在输出中包含停止符
)

from trl import GRPOConfig, GRPOTrainer
# 配置GRPO训练参数
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params, # 传入vLLM采样参数
    temperature = 1.0, # 生成时的温度，值越高，随机性越强
    learning_rate = 5e-6, # GRPO的学习率通常比SFT小
    weight_decay = 0.01,
    warmup_ratio = 0.1, # 预热步数占总步数的比例
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1, # 每一步都记录日志，便于观察
    per_device_train_batch_size = 1, # GRPO中，这个值通常会被num_generations覆盖
    gradient_accumulation_steps = 1, # 梯度累积，可以增加到4以获得更平滑的训练
    num_generations = 4, # 每个prompt生成4个不同的回答进行评估，如果显存不足可以减小此值
    max_prompt_length = max_prompt_length, # prompt最大长度
    max_completion_length = max_completion_length, # 生成内容最大长度
    # num_train_epochs = 1, # 训练总轮数
    max_steps = 100, # 为了快速演示，这里只训练100步
    save_steps = 100, # 每100步保存一次模型
    report_to = "swanlab", # 日志报告平台
    output_dir = "outputs", # 模型和日志输出目录
)

In [ ]:
# 实例化GRPOTrainer
trainer = GRPOTrainer(
    model = model, # 我们的基础模型
    processing_class = tokenizer, # 分词器
    reward_funcs = [ # 传入我们定义的所有奖励函数
        # match_format_exactly, # 这个函数被注释掉了，因为它太严格了
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args, # 传入训练配置
    train_dataset = dataset, # 训练数据集
)
# 开始GRPO训练
trainer.train()

In [ ]:
# GRPO训练后进行推理测试
text = "What is the sqrt of 101?"

from vllm import SamplingParams
# 定义推理时的采样参数
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)

# 使用Unsloth的fast_generate方法进行快速推理
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None, # 因为LoRA权重已经合并到模型中，所以这里是None
)[0].outputs[0].text

output

In [ ]:
# 将训练好的LoRA适配器权重保存到本地目录
model.save_lora("grpo_saved_lora")

In [ ]:
# 这是一个验证步骤，确保我们保存的LoRA权重是有效的。
from safetensors import safe_open

tensors = {}
# 使用safetensors库安全地打开保存的适配器模型文件
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # 遍历文件中的所有张量（tensors）
    for key in f.keys():
        tensor = f.get_tensor(key)
        # 计算张量中零元素的比例
        n_zeros = (tensor == 0).sum() / tensor.numel()
        # 断言：确保张量不全为零，证明训练确实改变了权重。
        assert(n_zeros.item() != tensor.numel())


In [ ]:
# 演示如何加载已保存的LoRA适配器进行推理
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

# 同样，先应用聊天模板
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)

# 在fast_generate中，通过lora_request参数加载指定的LoRA适配器
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

In [ ]:
# --- 模型保存与上传（GGUF格式） --- #
# 以下代码块展示了如何将微调后的模型保存为GGUF格式，并推送到HuggingFace Hub。
# GGUF是一种专为llama.cpp等推理框架设计的格式，支持量化，可以在CPU上高效运行。
# 所有代码都被if False:包裹，表示默认不执行。如需使用，请将False改为True。

# 保存为 8bit Q8_0 GGUF 格式（适用于GGUF量化模型推理）
if False:
    model.save_pretrained_gguf("model", tokenizer)

# 上传 Q8_0 量化模型到 HuggingFace Hub
# 请前往 https://huggingface.co/settings/tokens 获取你的访问令牌（token）
# 并将 "hf" 替换为你的HuggingFace用户名
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token="YOUR_HF_TOKEN")

# 保存为 16bit GGUF 格式（即未量化版本，保留完整精度，适用于对精度要求高的任务）
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")

# 上传 16bit GGUF 模型到 HuggingFace Hub
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method="f16", token="YOUR_HF_TOKEN")

# 保存为 q4_k_m（4位 K-type）GGUF 格式，这是一种在推理效率与模型精度之间取得良好平衡的常用量化方法。
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")

# 上传 q4_k_m 模型到 HuggingFace Hub
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method="q4_k_m", token="YOUR_HF_TOKEN")

# 如果你想一次性上传多个不同量化精度的GGUF版本，可以使用以下方法，速度更快。
if False:
    model.push_to_hub_gguf(
        "hf/model",  # 同样，替换 "hf" 为你的HuggingFace用户名
        tokenizer,
        quantization_method=["q4_k_m", "q8_0", "q5_k_m"],  # 可根据需要选择要上传的量化格式
        token="YOUR_HF_TOKEN"
    )
